# Intersection of polygons in 2D space

I need a simple method for detecting overlap between polygons in 2D space. For this I need:

- Points, (obviously)
- A Triangle, as the fastest way to determine how far a point is from a line, is by calculating the 0.5 * area / base of the triangle.
- A Line, as line-2-line intersection will tell me if a line from one polygon's boundary crosses anothers.
- A Circle to do fast evaluation of whether two polygons _could_ intersect
- and a Polygon to hold a collection of Points

Aside from these classes there's not much more to say really.

In [1]:
class Point:
    __slots__ = ['x','y']
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f"Point({self.x},{self.y})"
    def distance(self, other):
        return (abs(self.x - other.x) ** 2 + abs(self.y - other.y) ** 2) ** (1 / 2)
    def __iter__(self):
        return iter((self.x,self.y))
    def move(self,x,y):
        self.x+=x
        self.y+=y
        return self
    def __hash__(self) -> int:
        return hash((self.x,self.y))
    def __eq__(self, __value: object) -> bool:
        assert isinstance(__value, Point)
        return self.x==__value.x and self.y == __value.y

In [2]:
p1 = Point(1,2)
p2 = Point(2,1)

assert p1.distance(p2) == p2.distance(p1) != 0

p3 = Point(0,0)
assert p1 != p3
p3.move(1,2)
assert p1 == p3


In [3]:
class Triangle:
    def __init__(self, A, B, C) -> None:
        assert all(isinstance(i, Point) for i in (A,B,C))
        self.A, self.B, self.C = A,B,C
    
    @property
    def area(self):
        a, b, c = self.A, self.B, self.C
        return 1/2 * abs(a.x*(b.y-c.y) + b.x*(c.y-a.y) + c.x*(a.y-b.y))

    def __contains__(self, other):
        if isinstance(other, Point):
            a,b,c,p = self.A, self.B,self.C, other
            if other in [a,b,c]:
                return True
            
            area = self.area
            s = 1/(2*area)*(a.y*c.x - a.x*c.y + (c.y - a.y) * p.x + (a.x - c.x)*p.y)
            t = 1/(2*area)*(a.x*b.y - a.y*b.x + (a.y - b.y)*p.x + (b.x - a.x)*p.y)
            return s > 0 and t > 0 and 1-s-t>0

In [4]:
def cog(x,y):
    n = len(x)
    x = sum(x) / n
    y = sum(y) / n
    return Point(x, y)

In [5]:
t1 = Triangle(p1,p2, Point(2,2))
assert t1.area == 0.5, t1.area
x,y = zip(*[t1.A, t1.B,t1.C])
p4 = cog(x,y)
assert p4 in t1
assert p1 in t1
assert p2 in t1

In [6]:
class Line:
    def __init__(self, p1, p2) -> None:
        assert isinstance(p1, Point)
        assert isinstance(p2, Point)
        assert p1 != p2, p1
        self.p1 = p1
        self.p2 = p2
        self.length = p1.distance(p2)

    def __mul__(self, other):
        if not 0 <= other <= 1:
            raise ValueError("the line index should be in the range [0:1]")
        x = self.p1.x + (self.p2.x - self.p1.x) * other
        y = self.p1.y + (self.p2.y - self.p1.y) * other
        return Point(x, y)

    def intersect(self, other):
        """ https://bryceboe.com/2006/10/23/line-segment-intersection-algorithm/"""
        def ccw(A,B,C):
            return (C.y-A.y)*(B.x-A.x) > (B.y-A.y)*(C.x-A.x)

        A,B,C,D = self.p1,self.p2, other.p1, other.p2
        return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

    def distance(self, other):
        if isinstance(other, Point):  #A = 1/2h*B --> h = 2A/B
            return 2 * Triangle(self.p1,self.p2,other).area / self.length
        raise NotImplementedError()

In [7]:
a,b,c,d,e,f = Point(-1,0), Point(1,0), Point(0,1), Point(0,-1), Point(2,1), Point(2,-1)

line1 = Line(a,b)
assert line1.length == 2, line1.length
line2 = Line(c,d)
line3 = Line(e,f)
assert line1.intersect(line2)
assert not line1.intersect(line3)
assert line1.distance(Point(0,0)) == 0
assert line1.distance(Point(0,1)) == 1

In [8]:
class Circle:
    def __init__(self, center, radius) -> None:
        assert isinstance(center, Point)
        assert isinstance(radius, (float,int)) and radius >= 0
        self.center = center
        self.radius = radius

    def __contains__(self, point):
        return self.center.distance(point) <= self.radius

In [9]:
c = Circle(Point(0,0), radius=1)
assert Point(0,0) in c
assert Point(1,0) in c
assert Point(2,0) not in c

In [10]:
class Polygon(object):
    def __init__(self, *points) -> None:
        self.points = points
        self._hull = None
        self._cog = None
    def __repr__(self):
        return f"Polygon{self.points}"
    def move(self,x,y):
        for p in self.points:
            p.move(x,y)
        if self._cog:
            self._cog.move(x,y)
        
    @property   
    def area(self):
        """https://en.wikipedia.org/wiki/Shoelace_formula"""
        x, y = zip(*[(p.x, p.y) for p in self.hull])
        "Assumes x,y points go around the polygon in one direction"
        return abs(sum(x[i - 1] * y[i] - x[i] * y[i - 1] for i in range(len(x)))) / 2.0

    @property
    def cog(self):
        """center of gravity"""
        if self._cog is None:
            x, y = zip(*[(p.x, p.y) for p in self.hull])
            self._cog = cog(x,y)
        return self._cog        

    @property
    def hull(self):
        if self._hull is None:
            self._hull = self._convex_hull()
        return self._hull

    def _convex_hull(self):
        assert len(self.points) >= 3, "can't have a hull with less than 3 points."
        points = sorted(self.points, key=lambda p: (p.x, p.y))

        def right_turn(A, B, C):
            dxAC, dyAC = C.x - A.x, C.y - A.y
            dxAB, dyAB = B.x - A.x, B.y - A.y
            if dxAC == 0:
                aAC = float("inf")
            else:
                aAC = dyAC / dxAC
            if dxAB == 0:
                aAB = float("inf")
            else:
                aAB = dyAB / dxAB

            if aAB > aAC:  # right
                return True
            elif aAB < aAC:  # left
                return False
            else:  # straight
                return None

        upper_hull = []
        lower_hull = []
        for point in points:
            upper_hull.append(point)
            while len(upper_hull) >= 3:
                a, b, c = upper_hull[-3:]
                if right_turn(a, b, c) is True:
                    break
                else:
                    upper_hull.remove(b)

            lower_hull.append(point) 
            while len(lower_hull) >= 3:
                a, b, c = lower_hull[-3:]
                if right_turn(a, b, c) is False:
                    break
                else:
                    lower_hull.remove(b)

        return upper_hull[:-1] + lower_hull[::-1]

    def __contains__(self, other):
        if isinstance(other, Point):
            hull = self.hull
            cog = self.cog
            for a,b in zip(hull[-1:] + hull[:-1], hull):
                triangle = Triangle(a,b,cog)
                if other in triangle:
                    return True
            return False
        raise NotImplementedError()


    def intersects(self, other):
        if isinstance(other, Polygon):
            # quick checks:
            c1 = self.cog
            d1s = [c1.distance(p) for p in self.hull]
            c2 = other.cog
            d2s = [c2.distance(p) for p in other.hull]

            d = c1.distance(c2)
            if d > max(d1s) + max(d2s):  # d is greater than a circle around both polygons.
                return False
            if d < min(d1s) or d < min(d2s):  # d is smaller than the inner circle.
                return True
            
            # check if any segment cuts any other segment.
            for a,b in zip(self.hull[:-1], self.hull[1:]):
                for c,d in zip(other.hull[:-1], other.hull[1:]):
                    if Line(a,b).intersect(Line(c,d)):
                        return True
            return False
        raise NotImplementedError()


In [11]:
a,b,c,d,e,f = Point(-1,0), Point(1,0), Point(0,1), Point(0,-1), Point(2,1), Point(2,-1)
g,h,i,j = Point(0,0), Point(0,2), Point(2,2), Point(2,0)

pg1 = Polygon(a,b,c,d,e,f)
pg2 = Polygon(g,h,i,j)

In [12]:
print(pg1, "\n - cog: ", pg1.cog, "\n - area:", pg1.area)
print(pg2, "\n - cog: ", pg2.cog, "\n - area:",pg2.area)
print(pg1.intersects(pg2))

Polygon(Point(-1,0), Point(1,0), Point(0,1), Point(0,-1), Point(2,1), Point(2,-1)) 
 - cog:  Point(0.3333333333333333,0.0) 
 - area: 5.0
Polygon(Point(0,0), Point(0,2), Point(2,2), Point(2,0)) 
 - cog:  Point(0.8,0.8) 
 - area: 4.0
True


In [13]:
assert pg2.hull == [Point(0,0), Point(0,2), Point(2,2), Point(2,0), Point(0,0)]
pg2.move(0,1)
assert pg2.hull== [Point(0,1), Point(0,3), Point(2,3), Point(2,1), Point(0,1)]
assert pg2.cog == Point(0.8,1.8)
assert pg1.intersects(pg2), "edges touch!"
pg2.move(0,1)
assert pg2.cog == Point(0.8,2.8)
assert pg2.hull == [Point(0,2), Point(0,4), Point(2,4), Point(2,2), Point(0,2)]
assert not pg1.intersects(pg2)


Next I'm placing a tiny square inside pg2:

In [14]:
pg3 = Polygon(*[Point(*ab) for ab in [(1,3), (1,3.1), (1.1, 3.1), (1.1,3)]])
pg3

Polygon(Point(1,3), Point(1,3.1), Point(1.1,3.1), Point(1.1,3))

In [15]:
pg2.intersects(pg3)

True

In short: It works.